## 한국어 BERT의 다음 문장 예측(Next Sentence Prediction)
모든 BERT 실습은 Colab에서 진행한다고 가정합니다.
사전 학습된 한국어 BERT를 이용하여 다음 문장 예측을 실습해봅시다. 이번 실습을 위해서만이 아니라 앞으로 사전 학습된 BERT를 사용할 때는 transformers라는 패키지를 자주 사용하게 됩니다. 실습 환경에 transformers 패키지를 설치해둡시다.
```
pip install transformers
```

### 1. 다음 문장 예측 모델과 토크나이저
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [1]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

2022-04-12 13:13:02.258234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-12 13:13:02.258271: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TFBertForNextSentencePrediction.from_pretrained('BERT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드합니다.

AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [2]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

2022-04-12 13:21:31.220825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-12 13:21:31.220864: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-12 13:21:31.220877: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-41K4BMT): /proc/driver/nvidia/version does not exist
2022-04-12 13:21:31.221063: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSen

### 2. BERT의 입력
다음 문장 예측을 위해 문맥 상으로 실제로 이어지는 두 개의 문장을 준비합니다.

In [3]:
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."

앞서 준비한 klue/bert-base의 토크나이저를 사용하여 두 개의 문장을 정수 인코딩해봅시다.

In [4]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인할 수 있습니다.

In [5]:
print(encoding['input_ids'])

tf.Tensor(
[[    2  5791  2440  5339  4713  2104  2124  2259  3708  2145  3965  6233
   4414  2496  2359  2414  3665 31221  1751  9900 12190    18     3  4062
   2069   543  2178  2209  3629  2079  5791  2440  5339  4713  2104  7288
   3864  2259  5537  2371  2219  3606    18     3]], shape=(1, 44), dtype=int32)


여기서 주의할 점은 여기서 2와 3은 특별 토큰이라는 점입니다. 실제로 해당 토크나이저의 [CLS] 토큰과 [SEP] 토큰의 번호를 출력해봅시다.

In [6]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)

[CLS] : 2
[SEP] : 3


위의 정수 인코딩 결과를 다시 디코딩해보면 현재 입력의 구성을 확인할 수 있습니다.

In [7]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] 2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다. [SEP] 여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다. [SEP]


BERT에서 두 개의 문장이 입력으로 들어갈 경우에는 맨 앞에는 [CLS] 토큰이 존재하고, 첫번째 문장이 끝나면 [SEP] 토큰, 그리고 두번째 문장이 종료되었을 때 다시 추가적으로 [SEP] 토큰이 추가됩니다. 토크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있습니다.

In [8]:
print(encoding['token_type_ids'])

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1]], shape=(1, 44), dtype=int32)


0이 연속적으로 등장하다가 어느 순간부터 1이 연속적으로 등장하는데, 이는 [CLS] 토큰의 위치부터 첫번째 문장이 끝나고나서 등장한 [SEP] 토큰까지의 위치에는 0이 등장하고, 다음 두번째 문장부터는 1이 등장하는 것입니다. token_type_ids에서는 0과 1로 두 개의 문장을 구분하고 있습니다.

### 3. 다음 문장 예측하기
이제 TFBertForNextSentencePrediction를 통해서 다음 문장을 예측해봅시다. 모델에 입력을 넣으면, 해당 모델은 소프트맥스 함수를 지나기 전의 값인 logits을 리턴합니다. 해당 값을 소프트맥스 함수를 통과시킨 후 두 개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록 더 큰 확률값을 가진 인덱스를 리턴하도록 합니다.
  
BERT로 다음 문장 예측을 학습하여
 - 이어지는 두 개의 문장의 레이블은 0. 
 - 이어지지 않는 두 개의 문장의 경우에는 레이블을 1로
  
이진 분류로 학습된 것을 사용합니다.

In [9]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


최종 예측 레이블은 0입니다. 이는 BERT가 다음 문장 예측을 학습했을 당시에 실질적으로 이어지는 두 개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1로 두고서 이진 분류로 학습을 하였기 때문입니다. 
  
이번에는 이어지지 않는 두 개의 문장으로 테스트해봅시다.

In [10]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]
